# HashiCorp Vault Demo for KMIP Secrets Engine and MongoDB Client-Side Field Level Encryption (CSFLE) & Queryable Encryption (QE)

With MongoDB’s Client-Side Field Level Encryption (CSFLE) and Queryable Encryption (QE), applications can encrypt sensitive plain text fields in documents prior to transmitting data to the server. This means that data processed by database (in use) will not be in plain text as it’s always encrypted and most importantly still can be queried (QE). 

This demo showcases how MongoDB CSFLE and QE uses HashiCorp Vault's KMIP Secrets Engine to manage encryption keys via KMIP protocol.

Note: MongoDB Community & Enterprise server editions have the same developer features (queries, aggregation, replication, sharding, etc).  MongoDB Enterprise Advanced adds:
- Operational & security features (In-memory storage engine, Auditing, Kerberos & LDAP authentication, Encryption at Rest). https://www.mongodb.com/docs/manual/administration/upgrade-community-to-enterprise/
- Entitlements to use additional tools including Ops Manager, BI Connector, and Enterprise Operator for Kubernetes.

CSFLE is offered on both Community and Enterprise Edition. The Enterprise edition can perform automatic encryption based on JSONSchema (which is also used by the mongocryptd library). For the community edition the application will be responsible for performing this encryption. 
- https://www.mongodb.com/docs/manual/core/csfle/reference/compatibility/
- https://www.mongodb.com/docs/manual/core/queryable-encryption/reference/compatibility/

<img src="images/vault-demo-kmip-mongodb-csfle-qe.png">

## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup.  The components used in this demo are:
- Vault Enterprise installed on docker (to simulate an external Vault)
- MongoDB Enterprise installed on docker (to simulate an external MongoDB)
- You have the Vault CLI installed

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables.

Note: You will need Vault to be installed with an ADP KMIP license add-on.  Also note that a Premium license is required if you wish the KMIP listener to scale on the Vault cluster.  For Standard or Plus license, the KMIP listener is only on the leader node.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

In [ ]:
# Make sure Python 3 is installed for the tutorial python scripts to run
brew install python3

In [28]:
# Install python dependencies for the CSFLE/QE python script samples
python3 -m pip install pymongo
python3 -m pip install 'pymongo[encryption]'

  Using cached pymongo-4.6.2-cp311-cp311-macosx_10_9_universal2.whl.metadata (22 kB)
Using cached pymongo-4.6.2-cp311-cp311-macosx_10_9_universal2.whl (534 kB)

  Using cached pymongocrypt-1.8.0-py3-none-macosx_11_0_universal2.whl.metadata (21 kB)
  Using cached pymongo_auth_aws-1.1.0-py2.py3-none-any.whl.metadata (4.9 kB)
Using cached pymongocrypt-1.8.0-py3-none-macosx_11_0_universal2.whl (6.9 MB)
Using cached pymongo_auth_aws-1.1.0-py2.py3-none-any.whl (11 kB)



In [ ]:
# Optional: To upgrade current version
python3 -m pip install --upgrade pymongo
python3 -m pip install --upgrade 'pymongo[encryption]'


In [27]:
# Optional: To uninstall pymongo
python3 -m pip uninstall -y pymongo
python3 -m pip uninstall -y pymongo-auth-aws
python3 -m pip uninstall -y pymongocrypt

Found existing installation: pymongo 4.6.2
Uninstalling pymongo-4.6.2:
  Successfully uninstalled pymongo-4.6.2
Found existing installation: pymongo-auth-aws 1.1.0
Uninstalling pymongo-auth-aws-1.1.0:
  Successfully uninstalled pymongo-auth-aws-1.1.0
Found existing installation: pymongocrypt 1.8.0
Uninstalling pymongocrypt-1.8.0:
  Successfully uninstalled pymongocrypt-1.8.0


In [25]:
# Optional: Check pymongo version that you are using
python3 -m pip list | grep pymongo

: 1

# Setting up HashiCorp Vault

In [1]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
# Expose both the Vault API and the KMIP ports to the host machine.
# We will be using 5697 instead of 5696 for the KMIP port as the CSFLE/QE samples are already configured for that port
export VAULT_PORT=8200
export VAULT_KMIP_PORT=5697
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_KMIP_PORT}:${VAULT_KMIP_PORT} \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

14ffe0b5cb8a98a87de8d7cdf3a7aea86e8101de65c401b392f92ae2d70ecb35


# Setting up MongDB Enterprise

Note: Queryable Encryption does not work on a standalone instance.  Needs either a replica set or a sharded cluster.  For this demo, we will be using a single node replica set.
Ref: https://www.mongodb.com/community/forums/t/i-found-an-error-when-trying-to-use-mongodbs-queryable-encryption/188433/4

In [2]:
# Run MongoDB Enterprise in docker.  Expose the MongoDB port to the host machine. 
export MONGODB_PORT=27017
# Use latest 7.0 MongoDB Enterprise docker image.  This supports both linux/amd64 and linux/arm64
export MONGODB_TAG=7.0-ubuntu2204
# For UBI images
#export MONGODB_TAG=7.0-ubi8

docker run -d --rm -p ${MONGODB_PORT}:${MONGODB_PORT} --name mongodb-enterprise \
mongodb/mongodb-enterprise-server:$MONGODB_TAG mongod --replSet rs0 --port $MONGODB_PORT  \
--bind_ip "0.0.0.0"

222506d934ef55063b97fd5a53a8a043d2cd0d12cb3adf9efdfe710233947a27


In [4]:
# Initialize the replica set with a default configuration set
# Ref: https://www.mongodb.com/docs/manual/reference/method/rs.initiate/
docker exec -it mongodb-enterprise mongosh --eval "rs.initiate()"

]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000Current Mongosh Log ID:	65d8426e4b3313c62d457d16
Connecting to:		mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.1.5
Using MongoDB:		7.0.5
Using Mongosh:		2.1.5

For mongosh info see: https://docs.mongodb.com/mongodb-shell/


To help improve our products, anonymous usage data is collected and sent to MongoDB periodically (https://www.mongodb.com/legal/privacy-policy).
You can opt-out by running the disableTelemetry() command.

 ------
   The server generated these startup warnings when booting
   2024-02-23T06:59:52.917+00:00: Using the XFS filesystem is strongly recommended with the WiredTiger storage engine. See http://dochub.mongodb.org/core/prodnotes-filesystem
   2024-02-23T06:59:53.601+00:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
   2024-02-23T06:59:53.601+00:00: /sys/k

In [5]:
# View the status of your replica set
docker exec -it mongodb-enterprise mongosh --eval "rs.status()"

]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000Current Mongosh Log ID:	65d842718e5e95b7b2f9a714
Connecting to:		mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.1.5
Using MongoDB:		7.0.5
Using Mongosh:		2.1.5

For mongosh info see: https://docs.mongodb.com/mongodb-shell/

 ------
   The server generated these startup warnings when booting
   2024-02-23T06:59:52.917+00:00: Using the XFS filesystem is strongly recommended with the WiredTiger storage engine. See http://dochub.mongodb.org/core/prodnotes-filesystem
   2024-02-23T06:59:53.601+00:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
   2024-02-23T06:59:53.601+00:00: /sys/kernel/mm/transparent_hugepage/enabled is 'always'. We suggest setting it to 'never'
   2024-02-23T06:59:53.601+00:00: vm.max_map_count is too low
------

{
  set: 'rs0',
  date: ISODate('2024-02-23T07:00:01

# Setup MongoDB CSFLE/QE Sample

This section demonstrates the KMIP secrets engine and how it functions as a KMIP server with MongoDB Client-Side Field Level Encryption (CSFLE) and Queryable Encryption (QE).

Ref:
- https://developer.hashicorp.com/vault/docs/secrets/kmip
- https://www.mongodb.com/developer/products/atlas/hashicorp-vault-kmip-secrets-engine-mongodb/
- https://github.com/mongodb-developer/mongodb-kmip-fle-queryable

In [6]:
# Clone the tutorial repository for CSFLE/QE samples
git clone https://github.com/mongodb-developer/mongodb-kmip-fle-queryable

# Store the sample path for use later
export SAMPLE_PATH=./mongodb-kmip-fle-queryable/kmip-with-hashicorp-key-vault

Cloning into 'mongodb-kmip-fle-queryable'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 54 (delta 14), reused 48 (delta 10), pack-reused 0
Receiving objects: 100% (54/54), 1.86 MiB | 7.75 MiB/s, done.
Resolving deltas: 100% (14/14), done.


## Step 1 - Enable Vault KMIP Engine

In [7]:
# Enable KMIP secrets engine at the default mount path
#vault secrets disable kmip
vault secrets enable kmip

Success! Enabled the kmip secrets engine at: kmip/


In [8]:
# Start KMIP Server and set client TLS certificate TTL to 365 days

# Option 1 - Elliptic Curve (EC) is the default
vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT default_tls_client_ttl=365d
# Option 2 - RSA
#vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT tls_ca_key_type="rsa" tls_ca_key_bits=2048 default_tls_client_ttl=365d

# Check the KMIP configuration
echo
vault read kmip/config

Success! Data written to: kmip/config

Key                            Value
---                            -----
default_tls_client_key_bits    256
default_tls_client_key_type    ec
default_tls_client_ttl         8760h
listen_addrs                   [0.0.0.0:5697]
server_hostnames               [localhost]
server_ips                     [127.0.0.1 ::1]
tls_ca_key_bits                256
tls_ca_key_type                ec
tls_min_version                tls12


## Step 2 - Create KMIP Scope and Role in Vault

The KMIP secrets engine uses the concept of scopes to partition KMIP managed object storage into multiple named buckets. Within a scope, roles can be created which dictate the set of allowed operations that the particular role can perform. TLS client certificates can be generated for a role, which services and applications can then use when sending KMIP requests against Vault's KMIP secret engine.

In order to generate client certificates for KMIP clients to interact with Vault's KMIP server, we must first create a scope and role and specify the desired set of allowed operations for it.

In [9]:
# Settings for the KMIP scope
export KMIP_SVC_NAME=mongodb-svc

# Create a scope
vault write -f kmip/scope/$KMIP_SVC_NAME

Success! Data written to: kmip/scope/mongodb-svc


# Step 3 - Configure for CSFLE and Test

## Step 3a - Create KMIP role for CSFLE

In [10]:
# Settings for the KMIP role to be created
export KMIP_FLE_ROLE_NAME=fle

# Create FLE Role (note the TTL if not set, it uses the default Vault token/lease TTL of 768hrs)
vault write kmip/scope/$KMIP_SVC_NAME/role/$KMIP_FLE_ROLE_NAME tls_client_key_bits=2048 tls_client_key_type=rsa operation_all=true tls_client_ttl=365d

# List roles and verify the KMIP FLE role has been created
echo
vault list kmip/scope/$KMIP_SVC_NAME/role

Success! Data written to: kmip/scope/mongodb-svc/role/fle

Keys
----
fle


In [11]:
# Optional - View the FLE role details
vault read kmip/scope/$KMIP_SVC_NAME/role/$KMIP_FLE_ROLE_NAME

Key                    Value
---                    -----
operation_all          true
tls_client_key_bits    2048
tls_client_key_type    rsa
tls_client_ttl         8760h


## Step 3b - Create the KMIP certificates for MongoDB CSFLE

In [12]:
# Folder that is used by the CSFLE sample to store the KMIP certificates
echo "Sample Path: $SAMPLE_PATH"
export FLE_CERT_PATH=$SAMPLE_PATH/vault/certs/FLE

# Save KMIP CA certificate
vault read -format=json kmip/ca | jq -r .data.ca_pem > $FLE_CERT_PATH/vv-ca.pem    

# Create KMIP client certificates
vault write -format=json kmip/scope/$KMIP_SVC_NAME/role/$KMIP_FLE_ROLE_NAME/credential/generate format=pem > fle_credential.json
jq -r .data.certificate < fle_credential.json > $FLE_CERT_PATH/vv-cert.pem
jq -r .data.private_key < fle_credential.json > $FLE_CERT_PATH/vv-key.pem    
cat $FLE_CERT_PATH/vv-cert.pem $FLE_CERT_PATH/vv-key.pem > $FLE_CERT_PATH/vv-client.pem

# Cleanup temp json
rm fle_credential.json

Sample Path: ./mongodb-kmip-fle-queryable/kmip-with-hashicorp-key-vault


## Step 3c - Update the CSFLE Sample Connection String

In [13]:
# Replace this line in the configuration_fle.py file 
#   connection_uri = "mongodb+srv://<USER>:<PASSWORD>@<CLUSTER-NAME>?retryWrites=true&w=majority"
# with this
#   connection_uri = "mongodb://127.0.0.1:27017/admin?replicaSet=rs0&retryWrites=true&directConnection=true"
# Replace the connection string
# Escape "/"" with "\/"" and "&"" with "\&""
sed -i '' 's/mongodb+srv:\/\/<USER>:<PASSWORD>@<CLUSTER-NAME>?retryWrites=true&w=majority/mongodb:\/\/127.0.0.1:27017\/admin?replicaSet=rs0\&retryWrites=true\&directConnection=true/g' $SAMPLE_PATH/configuration_fle.py

# Show updated configuration_fle.py.  Verify that the connection_uri value has been updated.
cat $SAMPLE_PATH/configuration_fle.py

encrypted_namespace = "DEMO-KMIP-FLE.users"
key_vault_namespace = "DEMO-KMIP-FLE.datakeys"
connection_uri = "mongodb://127.0.0.1:27017/admin?replicaSet=rs0&retryWrites=true&directConnection=true"

# Configure the "kmip" provider.
kms_providers = {
    "kmip": {
        "endpoint": "localhost:5697"
    }
}
kms_tls_options = {
    "kmip": {
        "tlsCAFile": "vault/certs/FLE/vv-ca.pem",
        "tlsCertificateKeyFile": "vault/certs/FLE/vv-client.pem"
    }
}


## Step 3d - Test the CSFLE Sample

In [21]:
# Test the CSFLE sample.  This script demonstrates encrypt and decrypt functions with the HashiCorp Vault KMIP provider.
# Change to the script directory before running as the scripts are hard-code to find the client certs in /vault/certs/FLE
echo "Sample Path: $SAMPLE_PATH"
cd $SAMPLE_PATH
python3 vault_encrypt_with_csfle_kmip.py
cd ../..

Sample Path: ./mongodb-kmip-fle-queryable/kmip-with-hashicorp-key-vault
CSFLE Encryption: Decrypted document:
{'_id': ObjectId('65d8436dd4785a5f16678c77'),
 'address': {'city': 'Omaha', 'street': '123 Main', 'zip': '90210'},
 'contact': {'email': 'alan@example.com', 'mobile': '202-555-1212'},
 'firstName': 'Alan',
 'lastName': 'Turing',
 'ssn': '901-01-0001'}
CSFLE Encryption: Encrypted document:
{'_id': ObjectId('65d8436dd4785a5f16678c77'),
 'address': {'city': 'Omaha', 'street': '123 Main', 'zip': '90210'},
 'contact': {'email': Binary(b'\x02\x85T\xe2\xfa\x16MK\xdb\xb7\xc1A\xc9o\xe6\\\x95\x02}g\x95\xd8OHI<9\x12v\xe4\xdd\x1c\x81NPm\x8a\x92\x8e?kP\xb0H\x80\nw\x05\xcf\xae\xea0\xc0?\x9f<\x98\x841\x83U\x00\xbb\x91\xc6\x17\xce3\xcbu^\x12\xbeO\xae\xcc\xcf\xbf\xeb\x1bR\x1c$\x1f!\xc6\xbb\t\x01\x13\x1b\xf2\x0b\x18\x12\xb6v\x96', 6),
             'mobile': Binary(b'\x01\x85T\xe2\xfa\x16MK\xdb\xb7\xc1A\xc9o\xe6\\\x95\x02\xad\x97\xf7\x9d\x93\x88\xd7U#\xdf#\x8c\xc7\xa0/\xb3X\xedh\x94\xda\xbf;hQx\x

# Step 4 - Configure for QE and Test

## Step 4a - Create KMIP role for QE

In [ ]:
# Settings for the KMIP role to be created
export KMIP_QE_ROLE_NAME=qe

# Create FLE Role (note the TTL if not set, it uses the default Vault token/lease TTL of 768hrs)
vault write kmip/scope/$KMIP_SVC_NAME/role/$KMIP_QE_ROLE_NAME tls_client_key_bits=2048 tls_client_key_type=rsa operation_all=true tls_client_ttl=365d

# List roles and verify the QE KMIP role has been created
echo
vault list kmip/scope/$KMIP_SVC_NAME/role

In [ ]:
# Optional - View the QE role details
vault read kmip/scope/$KMIP_SVC_NAME/role/$KMIP_QE_ROLE_NAME

## Step 4b - Create the KMIP certificates for MongoDB QE

In [ ]:
# Folder that is used by the QE sample to store the KMIP certificates
echo "Sample Path: $SAMPLE_PATH"
export QE_CERT_PATH=$SAMPLE_PATH/vault/certs/QUERYABLE

# Save KMIP CA certificate
vault read -format=json kmip/ca | jq -r .data.ca_pem > $QE_CERT_PATH/vv-ca.pem    

# Create KMIP client certificates
vault write -format=json kmip/scope/$KMIP_SVC_NAME/role/$KMIP_QE_ROLE_NAME/credential/generate format=pem > qe_credential.json
jq -r .data.certificate < qe_credential.json > $QE_CERT_PATH/vv-cert.pem
jq -r .data.private_key < qe_credential.json > $QE_CERT_PATH/vv-key.pem    
cat $QE_CERT_PATH/vv-cert.pem $QE_CERT_PATH/vv-key.pem > $QE_CERT_PATH/vv-client.pem

# Cleanup temp json
rm qe_credential.json

## Step 4c - Update the QE Sample Connection String

In [ ]:
# Replace this line in the configuration_queryable.py file 
#   connection_uri = "mongodb+srv://<USER>:<PASSWORD>@<CLUSTER-NAME>?retryWrites=true&w=majority"
# with this
#   connection_uri = "mongodb://127.0.0.1:27017/admin?replicaSet=rs0&retryWrites=true&directConnection=true"
# Replace the connection string
# Escape "/"" with "\/"" and "&"" with "\&""
sed -i '' 's/mongodb+srv:\/\/<USER>:<PASSWORD>@<CLUSTER-NAME>?retryWrites=true&w=majority/mongodb:\/\/127.0.0.1:27017\/admin?replicaSet=rs0\&retryWrites=true\&directConnection=true/g' $SAMPLE_PATH/configuration_queryable.py

# Show updated configuration_fle.py.  Verify that the connection_uri value has been updated.
cat $SAMPLE_PATH/configuration_queryable.py

## Step 4d - Test the QE Sample

In [ ]:
# Test the CSFLE sample.  This script demonstrates encrypt and decrypt functions with the HashiCorp Vault KMIP provider.
# Change to the script directory before running as the scripts are hard-code to find the client certs in /vault/certs/FLE
echo "Sample Path: $SAMPLE_PATH"
cd $SAMPLE_PATH
python3 vault_encrypt_with_queryable_kmip.py
cd ../..

# Cleanup

In [ ]:
# Cleanup

# Disable KMIP secrets engine
vault secrets disable kmip

# Stop Vault container
docker stop vault-enterprise

# Stop MongoDB container
docker stop mongodb-enterprise

# Remove KMIP certificate demo files
rm -rf mongodb-kmip-fle-queryable